In [1]:
# ✅ 기능 점검 체크리스트
# 항목	적용 여부	설명
# 배송/회수 파일 병합	✅	read_file() + concat() 사용
# 날짜+시간 생성 및 정렬	✅	날짜+시간 컬럼 생성 후 정렬
# 하나의 방문코드 시퀀스 생성	✅	A0000001, A0000002 형식
# 중복 자전거번호 → 순번/flag 처리	✅	drop_duplicates() + "가짜" 표시
# 방문코드별 자전거대수, first/last	✅	.groupby().transform() 사용
# No는 시간순으로 전체 행에 일괄 부여	✅	df.sort_values(...).reset_index() 후 range()
# 요약 테이블 생성	✅	방문코드 기준 대표 대여소 + 레코드수
# CSV 및 요약 엑셀 저장	✅	to_csv() + to_excel() 처리

# ✅ 통합 최종 코드 (정렬 포함)
# 대여소를 문자열 기준으로 정렬, 방문코드 생성 / 중복 자전거 처리 / No 부여 / 요약 정리까지 완성된 최종 전체 코드

In [2]:
import pandas as pd
import os

# 1. 두 파일 경로
file_delivery = r"D:\pythondata\★_재배치_배송.csv"
file_return = r"D:\pythondata\★_재배치_회수.csv"

# 2. CSV 읽기 함수
def read_file(path, gubun_value):
    df = pd.read_csv(
        path,
        sep="\t",
        encoding="utf-8-sig",
        dtype={"대여소": str}
    )
    df["구분"] = gubun_value
    return df

# 3. 배송/회수 데이터프레임 생성
df_delivery = read_file(file_delivery, "배송")
df_return = read_file(file_return, "회수")

# 4. 병합
df = pd.concat([df_delivery, df_return], ignore_index=True)

# 5. 날짜+시간 생성
df["날짜+시간"] = pd.to_datetime(df["날짜"] + " " + df["시간"])

# 6. 대여소 문자열 기준 정렬 + 날짜+시간
df = df.sort_values(by=["대여소", "날짜+시간"]).reset_index(drop=True)

# 7. 방문코드 생성 (대여소 or 구분 or 30분 초과 시 변경)
visit_code = []
visit_index = 1
visit_code.append(f"A{visit_index:07d}")

for i in range(1, len(df)):
    time_diff = df.loc[i, "날짜+시간"] - df.loc[i - 1, "날짜+시간"]
    location_changed = df.loc[i, "대여소"] != df.loc[i - 1, "대여소"]
    type_changed = df.loc[i, "구분"] != df.loc[i - 1, "구분"]
    if time_diff.total_seconds() > 1800 or location_changed or type_changed:
        visit_index += 1
    visit_code.append(f"A{visit_index:07d}")

df["방문코드"] = visit_code

# 8. 순번 부여 (중복 자전거번호 고려)
unique_pairs = df.drop_duplicates(subset=["방문코드", "자전거번호"]).copy()
unique_pairs = unique_pairs.sort_values(by=["방문코드", "날짜+시간"])
unique_pairs["순번"] = unique_pairs.groupby("방문코드").cumcount() + 1

# 순번 병합
df = pd.merge(
    df.drop(columns=["순번"], errors="ignore"),
    unique_pairs[["방문코드", "자전거번호", "순번"]],
    on=["방문코드", "자전거번호"],
    how="left"
)

# 9. 중복 자전거번호 → flag
dup_mask = df.duplicated(subset=["방문코드", "자전거번호"], keep="first")
df["flag"] = ""
df.loc[dup_mask, "flag"] = "가짜"

# 10. 자전거대수 및 first/last 시간
df["자전거대수"] = df.groupby("방문코드")["대여소"].transform("count")
df["first data&time"] = pd.NaT
df["last data&time"] = pd.NaT
df.loc[df["순번"] == 1, "first data&time"] = df["날짜+시간"]
df.loc[df["순번"] == df["자전거대수"], "last data&time"] = df["날짜+시간"]

# 11. No 부여 (전체 시간 순 정렬)
df = df.sort_values(by=["날짜+시간"]).reset_index(drop=True)
df["No"] = range(1, len(df) + 1)

# 12. 컬럼 순서: No 맨 앞, 구분 맨 뒤
cols = df.columns.tolist()
cols = ["No"] + [col for col in cols if col not in ("No", "구분")] + ["구분"]
df = df[cols]

# 13. 요약 테이블 생성
station_info = df.groupby("방문코드")["대여소"].first().reset_index()
record_counts = df.groupby("방문코드")["대여소"].count().reset_index(name="레코드수")
summary = pd.merge(station_info, record_counts, on="방문코드")
summary = summary[["대여소", "방문코드", "레코드수"]]
summary = summary.sort_values(by="방문코드").reset_index(drop=True)

# 14. 저장
output_csv_path = r"D:\pythondata\5_통합_방문코드_중복_전체데이터.csv"
df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

output_summary_path = r"D:\pythondata\5_통합_방문코드_중복_요약.xlsx"
with pd.ExcelWriter(output_summary_path, engine="openpyxl") as writer:
    summary.to_excel(writer, sheet_name="요약", index=False)

print("✅ 통합 CSV 저장 완료:", output_csv_path)
print("✅ 통합 요약 엑셀 저장 완료:", output_summary_path)

✅ 통합 CSV 저장 완료: D:\pythondata\5_통합_방문코드_중복_전체데이터.csv
✅ 통합 요약 엑셀 저장 완료: D:\pythondata\5_통합_방문코드_중복_요약.xlsx
